In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
df = pd.read_csv('../../data/cohort_eICU_lac1.csv')
df.keys()

Index(['patient_id', 'sepsis3', 'hospital_id', 'icu_id', 'patient_id.1',
       'stay_id', 'stay_number', 'year', 'age', 'sex_female', 'race',
       'race_group', 'charlson_ci', 'CCI_ranges', 'sofa_day1', 'SOFA_ranges',
       'admit_type', 'admit_source', 'cirrhosis_present',
       'heart_failure_present', 'ckd_stages', 'weight', 'lactate_day1',
       'lactate_freq_day1', 'lactate_day2', 'lactate_freq_day2',
       'hemoglobin_stay_min', 'mortality_in', 'los_icu_hours',
       'mech_vent_overall_yes', 'rrt_overall_yes', 'rrt_start_delta',
       'vasopressor_overall_yes', 'transfusion_overall_yes', 'transfusion_yes',
       'transfusion_units_day1', 'transfusion_units_day2',
       'fluids_overall_yes', 'fluids_yes', 'fluids_sum_day1',
       'fluids_sum_day2'],
      dtype='object')

In [17]:
# split the data into bins by lactate
# bins: All, 0-2, 2-4, 4-6, >6
lacs = df.lactate_day1
races = ["White", "Black"]
bins = [0, 2, 4, 6]
dfs = {}
for race in races:
    dfs[f"All {race}"] = df[df["race_group"]==race]
    for i in range(len(bins)-1):
        dfs[f"{bins[i]}-{bins[i+1]} {race}"] = df[df["race_group"]==race][(lacs >= bins[i]) & (lacs < bins[i+1])]
    dfs[f">6 {race}"] = df[df["race_group"]==race][lacs >= bins[-1]]
# sort the dictionary by key
dfs = {k: dfs[k] for k in sorted(dfs.keys())}

C:\Users\Luca\AppData\Local\Temp\ipykernel_45908\399305802.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfs[f"{bins[i]}-{bins[i+1]} {race}"] = df[df["race_group"]==race][(lacs >= bins[i]) & (lacs < bins[i+1])]
C:\Users\Luca\AppData\Local\Temp\ipykernel_45908\399305802.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfs[f">6 {race}"] = df[df["race_group"]==race][lacs >= bins[-1]]


In [18]:
# Create a table with the following columns:
# mortality_in, los_icu_hours, mech_vent_overall_yes, rrt_overall_yes, vasopressor_overall_yes, transfusion_overall_yes, fluids_overall_yes
# split the columns by race_group (WHITE and BLACK)
# the rows will be the bins of lactate_day1
table = pd.DataFrame(columns=["mortality_in", "los_icu_hours", "mech_vent_overall_yes", "rrt_overall_yes", "vasopressor_overall_yes", "transfusion_overall_yes", "fluids_overall_yes"], index=dfs.keys())
for key in dfs.keys():
    table.loc[key] = [dfs[key].mortality_in.mean(), dfs[key].los_icu_hours.mean(), dfs[key].mech_vent_overall_yes.mean(), dfs[key].rrt_overall_yes.mean(), dfs[key].vasopressor_overall_yes.mean(), dfs[key].transfusion_overall_yes.mean(), dfs[key].fluids_overall_yes.mean()]
table

,mortality_in,los_icu_hours,mech_vent_overall_yes,rrt_overall_yes,vasopressor_overall_yes,transfusion_overall_yes,fluids_overall_yes
0-2 Black,0.105144,274.381745,0.369138,0.146747,0.380484,0.18003,0.178517
0-2 White,0.103673,222.546723,0.352143,0.071735,0.41449,0.128469,0.128367
2-4 Black,0.145577,252.036842,0.390817,0.136618,0.464726,0.160134,0.227324
2-4 White,0.146536,228.541588,0.374656,0.070158,0.492695,0.122812,0.131636
4-6 Black,0.182109,268.250319,0.485623,0.092652,0.549521,0.220447,0.210863
4-6 White,0.19469,221.835539,0.429204,0.071199,0.579646,0.139582,0.124698
>6 Black,0.349614,317.598843,0.660668,0.172237,0.712082,0.236504,0.210797
>6 White,0.375993,237.135385,0.630185,0.170786,0.718888,0.207414,0.127096
All Black,0.158382,272.646509,0.427151,0.141241,0.468632,0.185807,0.201234
All White,0.156767,225.935142,0.397671,0.081621,0.490939,0.136268,0.128861
